In [ ]:
# Instalação das bibliotecas necessárias
!pip install ultralytics
!pip install git+https://github.com/ultralytics/ultralytics.git@main

In [ ]:
# Conecta com sua conta no Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Importa as bibliotecas utilizadas
import os
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import csv

In [ ]:
# Substitua pelo caminho onde se encontra este notebook
ROOT_DIR = '/content/gdrive/MyDrive/Hackaton'

In [ ]:
# Carrega um modelo utilizando YOLO
def loadModel(path):
  return YOLO(path)

In [ ]:
# Realiza a predição de uma imagem
def predict(image, model):
  return model(image, conf=0.4)

In [ ]:
# Exibe a imagem após a predição
def plotImage(results):
  img_inferred= results[0].plot()
  plt.figure(figsize=(10, 10))
  plt.imshow(img_inferred)
  plt.xticks([]), plt.yticks([])
  plt.show()

In [ ]:
# Define qual tipo de simulado está sendo usado e retorna o modelo correspondente
def simulados(image, model):
  modelo = loadModel(model)
  results = predict(image, modelo)
  names_dict = results[0].names
  probs = results[0].probs.data.tolist()
  tipo = names_dict[np.argmax(probs)]
  print(tipo)
  if(tipo == 'blue'):
    return YOLO(os.path.join(ROOT_DIR, 'modelos/modeloAzulLast.pt'))
  elif (tipo == 'green'):
    return YOLO(os.path.join(ROOT_DIR, 'modelos/modeloVerdeLast.pt'))
  else:
    return YOLO(os.path.join(ROOT_DIR, 'modelos/diversos.pt'))

In [ ]:
# Analisa as detecções e retorna um dicionário ordenado pela questão e sua respectiva resposta
def resposta(image, modelo):
  results = predict(image, modelo)
  plotImage(results)
  names_dict = results[0].names

  for r in results:
    classes = r.boxes.cls.tolist()
    boxes = r.boxes.xywh.tolist()
  classe = []
  for item in classes:
    classe.append(names_dict[int(item)])
  for i in range(0, len(boxes)):
    boxes[i].append(classe[i])

# Ordena os boxes primeiro pelo eixo X e em seguida pelo eixo Y
  lista = []
  while len(boxes) > 0:
    x_min = min(item[0] for item in boxes)
    ordenado_x = []
    for item in boxes:
      if(item[0] > x_min - 50 and item[0] < x_min + 50):
        ordenado_x.append(item)
    ordem = sorted(ordenado_x, key=lambda x: x[1])
    boxes = [x for x in boxes if x not in ordenado_x]
    lista.append(ordem)

  respostas = {}
  i = 0
  for sublista in lista:
    for item in sublista:
      i += 1
      respostas[i] = item[4]
  return respostas

In [ ]:
# Realiza a comparação entre as respostas e o gabarito e retorna a quantidade de respostas certas
def resultado(gabarito, respostas):

  acertos = 0
  if len(gabarito) > len(respostas):
    for key, value in respostas.items():
      if gabarito[key] == value:
        acertos += 1
  else:
    for key, value in gabarito.items():
      if respostas[key] == value:
        acertos += 1

  return 'Você acertou: ' + str(acertos) + '/' + str(len(gabarito))

In [ ]:
# Diferencia se a imagem é uma prova ou um simulado
def provaSimulado(image, model):
  modelo = loadModel(model)
  results = predict(image, modelo)
  names_dict = results[0].names
  probs = results[0].probs.data.tolist()
  tipo = names_dict[np.argmax(probs)]
  if(tipo == 'prova'):
    modelo = YOLO(os.path.join(ROOT_DIR, 'modelos/prova.pt'))
    results = predict(image, modelo)
    names_dict = results[0].names
    for r in results:
      classes = r.boxes.cls.tolist()

    classe = []
    for item in classes:
      classe.append(names_dict[int(item)])

    acertos = 0
    if len(gabarito) > len(classe):
      for i in range(0, len(classe)):
        if gabarito[i] == classe[i]:
          acertos += 1
    else:
      for i in range(0, len(gabarito)):
        if gabarito[i] == classe[i]:
          acertos += 1

    return 'Você acertou: ' + str(acertos) + '/' + str(len(gabarito))

  else:
    modelo = os.path.join(ROOT_DIR, 'modelos/diferenciaSimuladosLast.pt')
    model = simulados(image, modelo)
    respostas = resposta(image, model)
    return respostas

In [ ]:
# Define o gabarito
def geraGabarito(image, model):
  modelo = loadModel(model)
  results = predict(image, modelo)
  names_dict = results[0].names
  probs = results[0].probs.data.tolist()
  tipo = names_dict[np.argmax(probs)]
  if(tipo == 'prova'):
    modelo = YOLO(os.path.join(ROOT_DIR, 'modelos/prova.pt'))
    results = predict(image, modelo)
    names_dict = results[0].names
    for r in results:
      classes = r.boxes.cls.tolist()

    classe = []
    for item in classes:
      classe.append(names_dict[int(item)])
    return classe
  else:
    modelo = os.path.join(ROOT_DIR, 'modelos/diferenciaSimuladosLast.pt')
    model = simulados(image, modelo)
    return resposta(image, model)


In [ ]:
def relatorio(gabarito, respostas):
  nome_arquivo = os.path.join(ROOT_DIR, 'relatorio.csv')

  # Abrir o arquivo CSV em modo de escrita
  with open(nome_arquivo, 'w', newline='') as arquivo_csv:
      # Criar o objeto de escrita CSV
      escritor_csv = csv.writer(arquivo_csv)

      # Escrever os dados do primeiro dicionário no CSV
      escritor_csv.writerow(['Questões'] + list(gabarito.keys()))
      escritor_csv.writerow(['Gabarito'] + list(gabarito.values()))

      # Escrever os dados do segundo dicionário no CSV
      escritor_csv.writerow(['Respostas'] + list(respostas.values()))
      lista = []
      if len(gabarito) > len(respostas):
        for i in range(1, len(respostas)+1):
          if gabarito[i] == respostas[i]:
            lista.append('Certo')
          else:
            lista.append('Errado')
      else:
        for i in range(1, len(respostas)+1):
          if gabarito[i] == respostas[i]:
            lista.append('Certo')
          else:
            lista.append('Errado')
      escritor_csv.writerow(['Comparação'] + lista)

In [ ]:
# Define o caminho do modelo inicial
pathModel = os.path.join(ROOT_DIR, 'modelos/provaGabarito.pt')

# Altere para o caminho da imagem que quer classificar
pathImageClassificar = os.path.join(ROOT_DIR, 'IMG_4508_JPG.rf.97aa730b7b0b6851d877ddd751d71b82.jpg')

# Altere para o caminho da imagem de gabarito
pathImageGabarito = os.path.join(ROOT_DIR, 'IMG_4515_JPG_jpg.rf.5002e1328a32bbf247c7b194f4143704.jpg')

gabarito = geraGabarito(pathImageGabarito, pathModel)
print('Gabarito: ' + str(gabarito))
respostas = provaSimulado(pathImageClassificar, pathModel)
print('Respostas: ' + str(respostas))
result = resultado(gabarito, respostas)
print(result)
relatorio(gabarito, respostas)